# **1. End-to-End Example with LangChain**

In [1]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
import os

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

template = """Bạn là 1 chat bot hữu ích và là 1 chuyên gia về lịch sử thế giới.
Hãy trả lời câu hỏi từ người dùng: {question} 
Lưu ý: bạn chỉ được sử dụng tiếng việt để trả lời."""
prompt = PromptTemplate(template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(llm=llm, prompt=prompt, memory=memory)

response = conversation({"question": "Ngày độc lập của nước Việt Nam là ngày nào ?"})
print("Question: ", response['question'])
print("History: ", response['chat_history'])
print("Answer: ", response['text'])

/home/ducpham/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/ducpham/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Question:  Ngày độc lập của nước Việt Nam là ngày nào ?
History:  
Answer:  Chào bạn! Tôi rất vui khi được trả lời câu hỏi của bạn.

Ngày độc lập của nước Việt Nam là ngày 2 tháng 9 năm 1945. Đây là ngày Chủ tịch Hồ Chí Minh đọc Tuyên ngôn Độc lập tại Quảng trường Ba Đình, Hà Nội, tuyên bố Việt Nam độc lập khỏi chế độ thực dân Pháp và thành lập Nhà nước Dân chủ Cộng hòa Việt Nam.


# **2. Memory Types in Langchain**

## **2.1 Conversation Buffer Memory**
Loại bộ nhớ này cho phép bạn lưu trữ và trích xuất tin nhắn từ các cuộc hội thoại

In [2]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

memory.save_context({"input": "Xin chào"}, 
                    {"output": "Có chuyện gì vậy ?"})
memory.save_context({"input": "bạn có phải là 1 chatbot không?"}, 
                    {"output": "Đúng vậy, tôi là 1 chatbot"})
history = memory.load_memory_variables({})
# Extract history as a string
print(history)


{'history': 'Human: Xin chào\nAI: Có chuyện gì vậy ?\nHuman: bạn có phải là 1 chatbot không?\nAI: Đúng vậy, tôi là 1 chatbot'}


## **2.2 Conversation Buffer Window Memory**
Loại bộ nhớ này giữ một danh sách các tương tác gần đây và sử dụng K tương tác cuối cùng, ngăn không cho bộ đệm trở nên quá lớn.

In [3]:
from langchain.memory import ConversationBufferWindowMemory


memory = ConversationBufferWindowMemory(k=1)

memory.save_context({"input": "Xin chào"}, 
                    {"output": "Có chuyện gì vậy ?"})
memory.save_context({"input": "bạn có phải là 1 chatbot không?"}, 
                    {"output": "Đúng vậy, tôi là 1 chatbot"})
history = memory.load_memory_variables({})
# Extract history as a string
print(history)

{'history': 'Human: bạn có phải là 1 chatbot không?\nAI: Đúng vậy, tôi là 1 chatbot'}


## **2.3 Conversation Entity Memory**
Loại bộ nhớ này ghi nhớ sự thật về các thực thể cụ thể trong cuộc trò chuyện và trích xuất thông tin bằng LLM.

In [4]:
from langchain.memory import ConversationEntityMemory


memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven và Sam đang cùng nhau làm 1 project trên Hackathon"}
memory.load_memory_variables(_input)
memory.save_context(_input, 
                    {"output": "Nghe có vẻ như một dự án tuyệt vời! Họ đang thực hiện loại dự án nào?"})

# _input = {"input": "Họ đang thực hiện 1 dự án về phân loại hình ảnh"}
# memory.load_memory_variables(_input)
# memory.save_context(_input, 
#                     {"output": "Thật tuyệt. Đó là 1 dự án hữu ích"})

history = memory.load_memory_variables({"input": 'Deven là ai ?'})
# Extract history as a string
print(history['history'])
print(history['entities'])


Human: Deven và Sam đang cùng nhau làm 1 project trên Hackathon
AI: Nghe có vẻ như một dự án tuyệt vời! Họ đang thực hiện loại dự án nào?
{'The output is: Deven': ''}


## **2.4 Conversation Knowledge Graph Memory**
Loại bộ nhớ này sử dụng biểu đồ tri thức để tạo lại bộ nhớ. Bạn có thể trích xuất các thực thể hiện tại và bộ ba kiến ​​thức từ tin nhắn.

In [5]:
from langchain.memory import ConversationKGMemory

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

memory = ConversationKGMemory(llm=llm)

memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})
memory.load_memory_variables({"input": "who is sam"})

{'history': ''}

## **2.5 Conversation Summary Memory**
Loại bộ nhớ này tạo ra bản tóm tắt cuộc trò chuyện theo thời gian, hữu ích cho việc cô đọng thông tin từ các cuộc trò chuyện dài.

In [6]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm)
memory.save_context({"input": "Xin chào"}, {"output": "Xin chào. Tôi có thể giúp gì cho bạn"})
memory.save_context({"input": "Bạn là 1 chatbot phải không ?"}, {"output": "Đúng vậy tôi là 1 trợ lý ảo. Hãy đưa ra thắc mắc, tôi sẽ giúp bạn trả lời."})
memory.load_memory_variables({})


{'history': 'Here is the new summary:\n\nThe human greets the AI in Vietnamese, saying "Xin chào". The AI responds with a greeting, "Xin chào", and asks how it can assist the human. The human asks if the AI is a chatbot, and the AI confirms it is a virtual assistant, offering to help answer any questions the human may have.'}

## **2.6 VectorStoreRetrieverMemory**

In [7]:
from datetime import datetime
from langchain.embeddings import FastEmbedEmbeddings
from langchain_groq import ChatGroq
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import faiss
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

embedding_size = 384 # dimension of the embedding model
index =faiss.IndexFlatL2(embedding_size)
embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5").embed_query
vector_store = FAISS(embedding_model, 
                     index,
                     InMemoryDocstore({}), {})

# Create VectorStoreRetrieverMemory
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
memory = VectorStoreRetrieverMemory(retriever=retriever)

# Save context and relevant information to the memory
memory.save_context({"input": "My favorite food is pizza"}, {"output": "that's good to know"})
memory.save_context({"input": "My favorite sport is soccer"}, {"output": "..."})
memory.save_context({"input": "I don't like the Celtics"}, {"output": "ok"})


print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


input: My favorite sport is soccer
output: ...


# **3. Combine memory with LLMChain**

In [7]:
import os
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain_groq import ChatGroq

# Create a ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện"
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will be injected
    ]
)

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = LLMChain(
    llm=llm, 
    prompt=prompt,
    verbose=True,
    memory=memory
)

response = chain.predict(human_input="Xin chào bạn của tôi.")
print(response)

/home/ducpham/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Human: Xin chào bạn của tôi.

> Finished chain.
Xin chào! Tôi rất vui được trò chuyện với bạn. Bạn cần giúp đỡ gì hoặc chỉ muốn tâm sự với tôi?


In [8]:
memory.buffer

[HumanMessage(content='Xin chào bạn của tôi.'),
 AIMessage(content='Xin chào! Tôi rất vui được trò chuyện với bạn. Bạn cần giúp đỡ gì hoặc chỉ muốn tâm sự với tôi?')]

In [9]:
response = chain.predict(human_input = "hãy nói tổng quan về Việt Nam")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Human: Xin chào bạn của tôi.
AI: Xin chào! Tôi rất vui được trò chuyện với bạn. Bạn cần giúp đỡ gì hoặc chỉ muốn tâm sự với tôi?
Human: hãy nói tổng quan về Việt Nam

> Finished chain.
Việt Nam! Đất nước tôi yêu quý!

Việt Nam là một nước Đông Nam Á, giáp với Trung Quốc phía bắc, Lào và Campuchia phía tây, biển Đông phía đông và nam. Đất nước chúng ta có hình chữ S, dài khoảng 1.700 km, rộng khoảng 500 km.

Việt Nam có diện tích khoảng 331.212 km², dân số hơn 98 triệu người, với hơn 54 dân tộc anh em, trong đó người Kinh chiếm đa số. Thủ đô của Việt Nam là Hà Nội, thành phố lớn nhất là Thành phố Hồ Chí Minh.

Việt Nam có một vị trí địa lý chiến lược, với bờ biển dài hơn 3.000 km, nhiều cảng biển quan trọng, và là một phần của con đường tơ lụa trên biển. Đất nước chúng ta có tài nguyên thiên nhiên khá phong phú

In [ ]:
memory.buffer

In [10]:
response = chain.predict(human_input = "Tôi vừa hỏi bạn điều gì ?")
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện
Human: Xin chào bạn của tôi.
AI: Xin chào! Tôi rất vui được trò chuyện với bạn. Bạn đang cần giúp đỡ gì hoặc chỉ muốn tám chuyện thư giãn?
Human: hãy nói tổng quan về Việt Nam
AI: Việt Nam là một nước đẹp và giàu truyền thống! Nước ta nằm ở Đông Nam Á, có biên giới với Trung Quốc, Lào, Campuchia, và biển Đông.

Việt Nam có diện tích khoảng 331.212 km², với hơn 98 triệu người, bao gồm 54 dân tộc khác nhau. Tiếng Việt là ngôn ngữ chính thức, và đa số người dân theo đạo Phật.

Nước ta có một lịch sử lâu đời và phong phú, với nhiều di tích lịch sử và văn hóa quý giá như Hoàng thành Thăng Long, Kinh thành Huế, và khu di tích Mỹ Sơn. Ngoài ra, Việt Nam còn có nhiều thắng cảnh thiên nhiên đẹp như Vịnh Hạ Long, Phong Nha - Kẻ Bàng, và Núi Bà Đen.

Việt Nam cũng là một nước có nền kinh tế đang phát triển, với tốc độ tă

# **4. Combine Prompt-LLM-Agent-Output_parser-Memory**

In [11]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import FastEmbedEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document``
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq
from groq import Groq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

csv_path = "/home/ducpham/Documents/VSC_PYTHON/RAG-Tutorial/Module_5_Memory/sample_csv.csv"

#load data
loader = CSVLoader("/home/ducpham/Documents/VSC_PYTHON/RAG-Tutorial/Module_2_Retrival/data/sample_csv.csv",
                   csv_args={
                       'delimiter': ',',
                        'quotechar': '"',
                        "fieldnames": ["name", "link", "tag"],
                   })

documents = loader.load()

# Split a long document into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents=documents)

# Create embedding
embedding_model = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5")
docsearch = Chroma.from_documents(
    texts, embedding_model
)

query = "Microservices là gì ?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

name: Microservices là gì?
link: https://viblo.asia/p/microservices-la-gi-gAm5yjD8Kdb
tag: microservices


In [21]:
print(docs)

[Document(page_content='name: Microservices là gì?\nlink: https://viblo.asia/p/microservices-la-gi-gAm5yjD8Kdb\ntag: microservices', metadata={'row': 25, 'source': '/home/ducpham/Documents/VSC_PYTHON/RAG-Tutorial/Module_2_Retrival/data/sample_csv.csv'}), Document(page_content='name: Microservice có gì hot? Chuyện về cái bồn cầu và microservice\nlink: https://toidicodedao.com/2017/02/21/tong-quan-micro-service/\ntag: toilet', metadata={'row': 590, 'source': '/home/ducpham/Documents/VSC_PYTHON/RAG-Tutorial/Module_2_Retrival/data/sample_csv.csv'}), Document(page_content='name: Microservices serial\nlink: https://viblo.asia/s/Wj53OQdw56m\ntag: unoconv', metadata={'row': 886, 'source': '/home/ducpham/Documents/VSC_PYTHON/RAG-Tutorial/Module_2_Retrival/data/sample_csv.csv'}), Document(page_content='name: Useful things to build microservice architecture\nlink: https://viblo.asia/s/o754jLDLZM6\ntag: opentracing', metadata={'row': 1139, 'source': '/home/ducpham/Documents/VSC_PYTHON/RAG-Tutorial

In [29]:
# Set up a prompt for the question-answering chain with memory
template = """Bạn là 1 chatbot hữu ích đang trò chuyện với người dùng.
Đưa ra các phần được trích xuất sau đây của một tài liệu dài và một câu hỏi, hãy tạo câu trả lời cuối cùng.
Lưu ý: bạn chỉ được sử dụng tiếng việt.
{chat_history}
Human: {human_input}
Chatbot:"""

from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

# Create a ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=
            """Bạn là một chatbot hữu ích đang trò chuyện với con người. 
            Từ phần context được cung cấp và câu hỏi từ người dùng hãy đưa ra câu trả lời cuối cùng. Dưới đây là context được cung cấp:
            {context}
            Lưu ý: bạn chỉ được sử dụng tiếng việt để trò chuyện"""
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will be injected
    ]
)

llm = ChatGroq(
    model="llama3-70b-8192",
    api_key= os.getenv("GROQ_API_KEY")
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


prompt[0].content = prompt[0].content.format(context = docs)


chain = LLMChain(
    llm=llm, 
    prompt=prompt,
    verbose=True,
    memory=memory
)

query = "Đưa ra các thông tin về Microservices."
response = chain.predict(human_input=query)
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: Bạn là một chatbot hữu ích đang trò chuyện với con người. 
            Từ phần context được cung cấp và câu hỏi từ người dùng hãy đưa ra câu trả lời cuối cùng. Dưới đây là context được cung cấp:
            [Document(page_content='name: Microservices là gì?\nlink: https://viblo.asia/p/microservices-la-gi-gAm5yjD8Kdb\ntag: microservices', metadata={'row': 25, 'source': '/home/ducpham/Documents/VSC_PYTHON/RAG-Tutorial/Module_2_Retrival/data/sample_csv.csv'}), Document(page_content='name: Microservice có gì hot? Chuyện về cái bồn cầu và microservice\nlink: https://toidicodedao.com/2017/02/21/tong-quan-micro-service/\ntag: toilet', metadata={'row': 590, 'source': '/home/ducpham/Documents/VSC_PYTHON/RAG-Tutorial/Module_2_Retrival/data/sample_csv.csv'}), Document(page_content='name: Microservices serial\nlink: https://viblo.asia/s/Wj53OQdw56m\ntag: unoconv', metadata={'row': 886, 'source': '/home/ducpham/Documents/VSC_PYTHON

In [36]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent
import os
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages

# Create a tool for searching
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")
search = TavilySearchResults()

tools = [
    Tool(
        name = "Search",
        func = search.run,
        description="hữu ích khi bạn cần trả lời câu hỏi về các sự kiện hiện tại"
    )
]
# Create a prompt with memory

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Bạn là một trợ lý ảo hữu ích. Bạn chỉ được trả lời bằng tiếng việt."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Bind tools
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x : x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        )
    }
    | prompt 
    | llm_with_tools 
    | OpenAIToolsAgentOutputParser() # cho ra output như các shell bên trên
)

agent_chain = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

# Ask a question and retrieve the answer
response = agent_chain.stream({"input": "model SORA của OpenAI có khả năng thế nào ?"})
print(list(response)[-1]['output'])




> Entering new AgentExecutor chain...

Invoking: `Search` with `model SORA của OpenAI có khả năng thế nào`


[{'url': 'https://openai.com/sora/', 'content': "Introducing Sora, our text-to-video model. Sora can generate videos up to a minute long while maintaining visual quality and adherence to the user's prompt. Prompt: A stylish woman walks down a Tokyo street filled with warm glowing neon and animated city signage. She wears a black leather jacket, a long red dress, and black boots, and carries ..."}, {'url': 'https://unite.ai/vi/những-gì-chúng-ta-biết-về-openais-sora-cho-đến-nay/', 'content': 'Mô hình Sora của OpenAI được coi là một thành tựu đáng chú ý trong quá trình phát triển thế hệ video AI, thể hiện những khả năng ấn tượng đồng thời nêu bật những thách thức và hạn chế đang diễn ra của công nghệ đó. Việc đưa nó vào bối cảnh AI nhấn mạnh tiềm ...'}, {'url': 'https://simplepage.vn/blog/sora-ai-tao-video/', 'content': 'Bên trên là toàn bộ những thông tin về Sora AI, một model t